In [1]:
import yaml
import time

import numpy as np
import pandas as pd

from Bio import Entrez

In [2]:
with open("apikeys.yaml", "r") as yamlfile:
  keys = yaml.load(yamlfile, Loader=yaml.FullLoader)
  print("Read Successful")

Read Successful


In [3]:
tcga = pd.read_csv("tcga_20190225Genes.csv")

In [4]:
tcga_gene = tcga[["symbol"]].dropna().drop_duplicates().values.tolist()

In [5]:
def get_pmid(ref_title, contact, key):
    ''' Using the Entrez search term, it queries the eSearch endpoint of the Entrez api to retrieve the corresponding pmids'''
    pmid = []
    counter = 0
    for i in range(len(ref_title)):
            Entrez.email = contact
            Entrez.api_key = key
            handle = Entrez.esearch(db='pubmed', term = ref_title[i], retmax=15)
            record = Entrez.read(handle)
            pmid.append(record['IdList'])

            if counter == 600:
                time.sleep(60)
                counter = 0
            
            counter += 1

    return pmid

In [6]:
tcga_pmids = get_pmid(tcga_gene, contact="rachit.sabharwal@uth.tmc.edu", key=keys["apikeys"]["ncbikey"]["key"])

In [9]:
tcga_pmids_dict = {"symbol":tcga_gene, "external_references_ncbi_pmids":tcga_pmids}
tcga_pmids_df = pd.DataFrame(tcga_pmids_dict, columns=["symbol", "external_references_ncbi_pmids"])

In [10]:
tcga_pmids_df.to_csv("tcga_pmids.csv", index=False)